<center>
<img src="https://laelgelcpublic.s3.sa-east-1.amazonaws.com/lael_50_years_narrow_white.png.no_years.400px_96dpi.png" width="300" alt="LAEL 50 years logo">
<h3>APPLIED LINGUISTICS GRADUATE PROGRAMME (LAEL)</h3>
</center>
<hr>

# Corpus Linguistics - Study 2 - Phase 1 - eyamrog

The aim of this phase is to web scrape the archive of mainstream journals in the areas of knowledge targeted in this study.

## Required Python packages

- beautifulsoup4
- lxml
- pandas
- requests
- tqdm

## Importing the required libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import sys
from tqdm import tqdm

## Defining input variables

In [2]:
input_directory = 'cl_st2_input_eyamrog'
output_directory = 'cl_st2_ph1_eyamrog'

## Creating output directory

In [3]:
# Check if the output directory already exists. If it does, do nothing. If it doesn't exist, create it.
if os.path.exists(output_directory):
    print('Output directory already exists.')
else:
    try:
        os.makedirs(output_directory)
        print('Output directory successfully created.')
    except OSError as e:
        print('Failed to create the directory:', e)
        sys.exit(1)

Output directory already exists.


## Inspecting the articles of the `chatgpt_erpp_pp` corpus per area of knowledge

### Importing the data into a DataFrame

In [4]:
df_scielo_preprint_preChatGPT_en = pd.read_json(f'{input_directory}/scielo_chatgpt_erpp_pp.jsonl', lines=True)

In [5]:
df_scielo_preprint_preChatGPT_en.dtypes

Title                    object
URL                      object
Authors                  object
Published                object
PDF Language             object
PDF URL                  object
Submitted                 int64
Posted                    int64
Text                     object
Text ID                  object
Area of Knowledge        object
Text Paragraphs          object
Text Paragraphs Count     int64
Text ChatGPT             object
dtype: object

In [6]:
df_scielo_preprint_preChatGPT_en['Submitted'] = pd.to_datetime(df_scielo_preprint_preChatGPT_en['Submitted'], unit='ms')
df_scielo_preprint_preChatGPT_en['Posted'] = pd.to_datetime(df_scielo_preprint_preChatGPT_en['Posted'], unit='ms')

In [7]:
df_scielo_preprint_preChatGPT_en

,Title,URL,Authors,Published,PDF Language,PDF URL,Submitted,Posted,Text,Text ID,Area of Knowledge,Text Paragraphs,Text Paragraphs Count,Text ChatGPT
0,"(Fern flora of Viçosa, Minas Gerais State, Bra...",https://preprints.scielo.org/index.php/scielo/...,"Nelson Túlio Lage Pena, Pedro Bond Schwartsburd",Submitted 11/22/2022 - Posted 11/23/2022,PDF,https://preprints.scielo.org/index.php/scielo/...,2022-11-22,2022-11-23,Publication status: Preprint has been publishe...,t000000,Biological Sciences,"(Fern flora of Viçosa, Minas Gerais State, Bra...",29,In this research article focusing on the fern ...
1,Assembling the perfect bacterial genome using ...,https://preprints.scielo.org/index.php/scielo/...,"Ryan R. Wick, Louise M. Judd, Kathryn E. Holt",Submitted 11/11/2022 - Posted 11/11/2022,PDF,https://preprints.scielo.org/index.php/scielo/...,2022-11-11,2022-11-11,Publication status: Preprint has been publishe...,t000001,Biological Sciences,A perfect bacterial genome assembly is one whe...,31,A flawless bacterial genome assembly is charac...
2,ON METHODOLOGY AND METHODS FOR ANALYSING CLASS...,https://preprints.scielo.org/index.php/scielo/...,Leonardo Goncalves Lago,Submitted 11/10/2022 - Posted 11/16/2022,PDF,https://preprints.scielo.org/index.php/scielo/...,2022-11-10,2022-11-16,Publication status: Preprint has been publishe...,t000002,Applied Social Sciences,This article presents a theoretical work whose...,65,This article presents a theoretical work with ...
3,"MOBILIZING LINGUISTIC AND SEMIOTIC RESOURCES, ...",https://preprints.scielo.org/index.php/scielo/...,"Estêvão Cabral, Marylin Martin-Jones",Submitted 11/07/2022 - Posted 11/07/2022,PDF,https://preprints.scielo.org/index.php/scielo/...,2022-11-07,2022-11-07,Publication status: Preprint has been submitte...,t000003,"Linguistic, literature and arts",This paper is based on research of a socioling...,58,This paper presents a sociolinguistic and ethn...
4,PORTUGUESE AO PÉ DO BERIMBAU: ON CAPOEIRA AS A...,https://preprints.scielo.org/index.php/scielo/...,"Mike Baynham, Jolana Hanusova",Submitted 11/04/2022 - Posted 11/04/2022,PDF,https://preprints.scielo.org/index.php/scielo/...,2022-11-04,2022-11-04,Publication status: Preprint has been submitte...,t000004,"Linguistic, literature and arts",From its historical origins as a resistant and...,123,Originating as a defiant and violently suppres...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,Challenges in the fight against the COVID-19 p...,https://preprints.scielo.org/index.php/scielo/...,Eduardo Alexandrino Servolo Medeiros,Submitted 04/15/2020 - Posted 04/15/2020,PDF,https://preprints.scielo.org/index.php/scielo/...,2020-04-15,2020-04-15,*Corresponding author. E-mail: edubalaccih@gma...,t000311,Health Sciences,We are living the most important pandemic in r...,9,"The ongoing global pandemic, triggered by the ..."
312,Information about the new coronavirus disease ...,https://preprints.scielo.org/index.php/scielo/...,Claudio Márcio Amaral de Oliveira Lima,Submitted 04/13/2020 - Posted 04/13/2020,PDF,https://preprints.scielo.org/index.php/scielo/...,2020-04-13,2020-04-13,V\nRadiol Bras. 2020 Mar/Abr;53(2):V–VI\n0100-...,t000312,Health Sciences,"Coronavirus is a zoonotic virus, an RNA virus ...",9,The coronavirus is a zoonotic RNA virus belong...
313,ACE2 diversity in placental mammals reveals th...,https://preprints.scielo.org/index.php/scielo/...,"Bibiana Sampaio de Oliveira Fam, Pedro Vargas-...",Submitted 04/11/2020 - Posted 04/28/2020,PDF,https://preprints.scielo.org/index.php/scielo/...,2020-04-11,2020-04-28,Status: Preprint has been published in a journ...,t000313,Biological Sciences,The recent emergence of SARS-CoV-2 is responsi...,35,The recent emergence of SARS-CoV-2 has led to ...
314,Coronavirus 2: Analysis of Regularity of Compl...,https://preprints.scielo.org/index.php/scielo/...,Yuri Morales-López,Submitted 04/10/2020 - Posted 06/04/2020,PDF,https://preprints.scielo.org/index.php/scielo/...,2020-04-10,20

### Agricultural Sciences

In [8]:
df_agricultural_sciences = df_scielo_preprint_preChatGPT_en[df_scielo_preprint_preChatGPT_en['Area of Knowledge'] == 'Agricultural Sciences']
df_agricultural_sciences[['Text ID', 'Text Paragraphs Count', 'Title']].to_csv(f'{output_directory}/scielo_agricultural_sciences.tsv', sep='\t', index=False, encoding='utf-8', lineterminator='\n')

### Applied Social Sciences

In [9]:
df_applied_social_sciences = df_scielo_preprint_preChatGPT_en[df_scielo_preprint_preChatGPT_en['Area of Knowledge'] == 'Applied Social Sciences']
df_applied_social_sciences[['Text ID', 'Text Paragraphs Count', 'Title']].to_csv(f'{output_directory}/scielo_applied_social_sciences.tsv', sep='\t', index=False, encoding='utf-8', lineterminator='\n')

### Biological Sciences

In [10]:
df_biological_sciences = df_scielo_preprint_preChatGPT_en[df_scielo_preprint_preChatGPT_en['Area of Knowledge'] == 'Biological Sciences']
df_biological_sciences[['Text ID', 'Text Paragraphs Count', 'Title']].to_csv(f'{output_directory}/scielo_biological_sciences.tsv', sep='\t', index=False, encoding='utf-8', lineterminator='\n')

### Engineering

In [11]:
df_engineering = df_scielo_preprint_preChatGPT_en[df_scielo_preprint_preChatGPT_en['Area of Knowledge'] == 'Engineering']
df_engineering[['Text ID', 'Text Paragraphs Count', 'Title']].to_csv(f'{output_directory}/scielo_engineering.tsv', sep='\t', index=False, encoding='utf-8', lineterminator='\n')

### Exact and Earth Sciences

In [12]:
df_exact_earth_sciences = df_scielo_preprint_preChatGPT_en[df_scielo_preprint_preChatGPT_en['Area of Knowledge'] == 'Exact and Earth Sciences']
df_exact_earth_sciences[['Text ID', 'Text Paragraphs Count', 'Title']].to_csv(f'{output_directory}/exact_earth_sciences.tsv', sep='\t', index=False, encoding='utf-8', lineterminator='\n')

### Health Sciences

In [13]:
df_health_sciences = df_scielo_preprint_preChatGPT_en[df_scielo_preprint_preChatGPT_en['Area of Knowledge'] == 'Health Sciences']
df_health_sciences[['Text ID', 'Text Paragraphs Count', 'Title']].to_csv(f'{output_directory}/scielo_health_sciences.tsv', sep='\t', index=False, encoding='utf-8', lineterminator='\n')

### Human Sciences

In [14]:
df_human_sciences = df_scielo_preprint_preChatGPT_en[df_scielo_preprint_preChatGPT_en['Area of Knowledge'] == 'Human Sciences']
df_human_sciences[['Text ID', 'Text Paragraphs Count', 'Title']].to_csv(f'{output_directory}/scielo_human_sciences.tsv', sep='\t', index=False, encoding='utf-8', lineterminator='\n')

### Linguistics, literature and arts

In [15]:
df_linguistics_literature_arts = df_scielo_preprint_preChatGPT_en[df_scielo_preprint_preChatGPT_en['Area of Knowledge'] == 'Linguistic, literature and arts']
df_linguistics_literature_arts[['Text ID', 'Text Paragraphs Count', 'Title']].to_csv(f'{output_directory}/scielo_linguistics_literature_arts.tsv', sep='\t', index=False, encoding='utf-8', lineterminator='\n')

## Agricultural Sciences

### Scraping [Annual Review of Plant Biology](https://www.annualreviews.org/content/journals/arplant)

### Scraping [Nature Food](https://www.nature.com/natfood/)

#### Capturing a few document pages for inspection

In [16]:
url_sample_1 =  'https://www.nature.com/natfood/research-articles?searchType=journalSearch&sort=PubDate&type=article&page=1'
url_sample_11 =  'https://www.nature.com/articles/s43016-024-01039-1'

In [17]:
document_page_sample_1 = requests.get(url_sample_1)
document_page_sample_11 = requests.get(url_sample_11)

In [18]:
with open(f'{output_directory}/natfood_sample1.html', 'wb') as file:
    file.write(document_page_sample_1.content)

with open(f'{output_directory}/natfood_sample11.html', 'wb') as file:
    file.write(document_page_sample_11.content)

#### Scraping the articles

In [19]:
root_url = 'https://www.nature.com'
article_list_url = 'https://www.nature.com/natfood/research-articles?searchType=journalSearch&sort=PubDate&type=article&page='
start_page = 1
end_page = 15
area_of_knowledge = 'Agricultural Sciences'

In [20]:
# Initialize an empty list to store the data
data = []

# Iterating through the URLs and using 'tqdm' for progress tracking in the range loop
for i in tqdm(range(start_page, end_page + 1)):
    url = f"{article_list_url}{i}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')

    # Finding all <article> elements with the class 'u-full-height c-card c-card--flush'
    articles = soup.find_all('article', class_='u-full-height c-card c-card--flush')

    for article in articles:
        # Extracting the title
        title_tag = article.find('h3', class_='c-card__title').find('a')
        title = title_tag.get_text(strip=True) if title_tag else ''
        title_url = f"{root_url}{title_tag.get('href')}" if title_tag else ''

        # Extracting the authors
        author_tags = article.find_all('li', itemprop='creator')
        authors = ', '.join(author.get_text(strip=True) for author in author_tags)

        # Extracting the published date
        date_tag = article.find('time')
        date_published = date_tag['datetime'] if date_tag else ''

        # Extracting the PDF URL
        pdf_url = f"{title_url}.pdf" if title_url else ''

        # Extracting the 'Open Access' label
        open_access_tag = article.find('span', class_='u-color-open-access')
        open_access = open_access_tag.get_text(strip=True) if open_access_tag else ''

        # Appending the data to the list
        data.append({
            'Title': title,
            'URL': title_url,
            'Authors (compact list)': authors,
            'Published': date_published,
            'PDF URL': pdf_url,
            'Open Access': open_access,
            'Area of Knowledge': area_of_knowledge
        })

# Creating a DataFrame from the data
df_nature_food = pd.DataFrame(data)

100%|██████████| 15/15 [00:18<00:00,  1.22s/it]


In [21]:
df_nature_food['Published'] = pd.to_datetime(df_nature_food['Published'])
df_nature_food

,Title,URL,Authors (compact list),Published,PDF URL,Open Access,Area of Knowledge
0,Potassium limits productivity in intensive cer...,https://www.nature.com/articles/s43016-024-010...,"Gonzalo Rizzo, Fahmuddin Agus, Patricio Grassini",2024-10-22,https://www.nature.com/articles/s43016-024-010...,,Agricultural Sciences
1,Proteomic signatures of healthy dietary patter...,https://www.nature.com/articles/s43016-024-010...,"Kai Zhu, Rui Li, Gang Liu",2024-09-27,https://www.nature.com/articles/s43016-024-010...,,Agricultural Sciences
2,Localized nitrogen management strategies can h...,https://www.nature.com/articles/s43016-024-010...,"Yize Liu, Minghao Zhuang, Yan Hao",2024-09-27,https://www.nature.com/articles/s43016-024-010...,,Agricultural Sciences
3,Human activities shape important geographic di...,https://www.nature.com/articles/s43016-024-010...,"Yuping Xiang, Guangliang Liu, Guibin Jiang",2024-09-26,https://www.nature.com/articles/s43016-024-010...,,Agricultural Sciences
4,Enhanced agricultural carbon sinks provide ben...,https://www.nature.com/articles/s43016-024-010...,"Stefan Frank, Andrey Lessa Derci Augustynczik,...",2024-09-23,https://www.nature.com/articles/s43016-024-010...,Open Access,Agricultural Sciences
...,...,...,...,...,...,...,...
277,Improved performance ofEimeria-infected chicke...,https://www.nature.com/articles/s43016-020-0029-6,"Philip A. Lessard, Matthew Parker, R. Michael ...",2020-02-10,https://www.nature.com/articles/s43016-020-002...,,Agricultural Sciences
278,Multidimensional characterization of global fo...,https://www.nature.com/articles/s43016-019-0012-2,"James Bentham, Gitanjali M Singh, Majid Ezzati",2020-01-13,https://www.nature.com/articles/s43016-019-001...,,Agricultural Sciences
279,Identification and stacking of crucial traits ...,https://www.nature.com/articles/s43016-019-0007-z,"Ratan Chopra, Evan B. Johnson, M. David Marks",2020-01-13,https://www.nature.com/articles/s43016-019-000...,,Agricultural Sciences
280,Shifts in national land use and food productio...,https://www.nature.com/articles/s43016-019-0011-3,"Paul D. L. Ritchie, Greg S. Smith, Ian J. Bateman",2020-01-13,https://www.nature.com/articles/s43016-019-001...,,Agricultural Sciences


#### Selecting the 'Open Access' articles published before the advent of ChatGPT

ChatGPT was released by OpenAI on November 30, 2022.

In [22]:
df_nature_food_open_access = df_nature_food[df_nature_food['Open Access'] == 'Open Access']
df_nature_food_open_access = df_nature_food_open_access[df_nature_food_open_access['Published'] <= '2022-11-30']
df_nature_food_open_access = df_nature_food_open_access.reset_index(drop=True)
df_nature_food_open_access

,Title,URL,Authors (compact list),Published,PDF URL,Open Access,Area of Knowledge
0,Transfer of cannabinoids into the milk of dair...,https://www.nature.com/articles/s43016-022-006...,"Bettina Wagner, Pietro Gerletti, Robert Pieper",2022-11-14,https://www.nature.com/articles/s43016-022-006...,Open Access,Agricultural Sciences
1,A systematic scoping review of the sustainabil...,https://www.nature.com/articles/s43016-022-006...,"A. Charlotte Bunge, Amanda Wood, Line J. Gordon",2022-11-03,https://www.nature.com/articles/s43016-022-006...,Open Access,Agricultural Sciences
2,Rights and representation support justice acro...,https://www.nature.com/articles/s43016-022-006...,"Christina C. Hicks, Jessica A. Gephart, Rosamo...",2022-10-18,https://www.nature.com/articles/s43016-022-006...,Open Access,Agricultural Sciences
3,Global dietary quality in 185 countries from 1...,https://www.nature.com/articles/s43016-022-005...,"Victoria Miller, Patrick Webb, Rubina Hakeem",2022-09-19,https://www.nature.com/articles/s43016-022-005...,Open Access,Agricultural Sciences
4,Food system by-products upcycled in livestock ...,https://www.nature.com/articles/s43016-022-005...,"Vilma Sandström, Anna Chrysafi, Matti Kummu",2022-09-19,https://www.nature.com/articles/s43016-022-005...,Open Access,Agricultural Sciences
5,Global food insecurity and famine from reduced...,https://www.nature.com/articles/s43016-022-005...,"Lili Xia, Alan Robock, Ryan Heneghan",2022-08-15,https://www.nature.com/articles/s43016-022-005...,Open Access,Agricultural Sciences
6,A shift from cattle to camel and goat farming ...,https://www.nature.com/articles/s43016-022-005...,"Jaber Rahimi, Erwann Fillol, Klaus Butterbach-...",2022-07-21,https://www.nature.com/articles/s43016-022-005...,Open Access,Agricultural Sciences
7,Time management governs climate resilience and...,https://www.nature.com/articles/s43016-022-005...,"Andrew J. McDonald, Balwinder-Singh, R. K. Malik",2022-07-21,https://www.nature.com/articles/s43016-022-005...,Open Access,Agricultural Sciences
8,A hybrid inorganic–biological artificial photo...,https://www.nature.com/articles/s43016-022-005...,"Elizabeth C. Hann, Sean Overa, Robert E. Jinke...",2022-06-23,https://www.nature.com/articles/s43016-022-005...,Open Access,Agricultural Sciences
9,A systematic review of the definitions and int...,https://www.nature.com/articles/s43016-022-005...,"Kajsa Resare Sahlin, Joanna Trewern",2022-06-23,https://www.nature.com/articles/s43016-022-005...,Open Access,Agricultural Sciences


#### Inspecting a few articles

In [23]:
df_nature_food_open_access.at[0, 'PDF URL']

'https://www.nature.com/articles/s43016-022-00623-7.pdf'

#### Exporting to a file

In [24]:
df_nature_food_open_access.to_json(f"{output_directory}/nature_food_open_access.jsonl", orient='records', lines=True)

### Scraping [Nature Sustainability](https://www.nature.com/natsustain/)

#### Scraping the articles

In [3]:
root_url = 'https://www.nature.com'
article_list_url = 'https://www.nature.com/natsustain/research-articles?searchType=journalSearch&sort=PubDate&page='
start_page = 1
end_page = 37
area_of_knowledge = 'Agricultural Sciences'

In [4]:
# Initialize an empty list to store the data
data = []

# Iterating through the URLs and using 'tqdm' for progress tracking in the range loop
for i in tqdm(range(start_page, end_page + 1)):
    url = f"{article_list_url}{i}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')

    # Finding all <article> elements with the class 'u-full-height c-card c-card--flush'
    articles = soup.find_all('article', class_='u-full-height c-card c-card--flush')

    for article in articles:
        # Extracting the title
        title_tag = article.find('h3', class_='c-card__title').find('a')
        title = title_tag.get_text(strip=True) if title_tag else ''
        title_url = f"{root_url}{title_tag.get('href')}" if title_tag else ''

        # Extracting the authors
        author_tags = article.find_all('li', itemprop='creator')
        authors = ', '.join(author.get_text(strip=True) for author in author_tags)

        # Extracting the published date
        date_tag = article.find('time')
        date_published = date_tag['datetime'] if date_tag else ''

        # Extracting the PDF URL
        pdf_url = f"{title_url}.pdf" if title_url else ''

        # Extracting the 'Open Access' label
        open_access_tag = article.find('span', class_='u-color-open-access')
        open_access = open_access_tag.get_text(strip=True) if open_access_tag else ''

        # Appending the data to the list
        data.append({
            'Title': title,
            'URL': title_url,
            'Authors (compact list)': authors,
            'Published': date_published,
            'PDF URL': pdf_url,
            'Open Access': open_access,
            'Area of Knowledge': area_of_knowledge
        })

# Creating a DataFrame from the data
df_nature_sustainability = pd.DataFrame(data)

100%|██████████| 37/37 [00:59<00:00,  1.61s/it]


In [5]:
df_nature_sustainability['Published'] = pd.to_datetime(df_nature_sustainability['Published'])
df_nature_sustainability

,Title,URL,Authors (compact list),Published,PDF URL,Open Access,Area of Knowledge
0,Hybrid polymer network cathode-enabled soluble...,https://www.nature.com/articles/s41893-024-014...,"Meng Liao, Yaobin Xu, Donghai Wang",2024-10-22,https://www.nature.com/articles/s41893-024-014...,,Agricultural Sciences
1,Sustainable lithium extraction and magnesium h...,https://www.nature.com/articles/s41893-024-014...,"Ming Yong, Meng Tang, Huanting Wang",2024-10-22,https://www.nature.com/articles/s41893-024-014...,,Agricultural Sciences
2,Radical climate protests linked to increases i...,https://www.nature.com/articles/s41893-024-014...,"Markus Ostarek, Brent Simpson, James Ozden",2024-10-21,https://www.nature.com/articles/s41893-024-014...,,Agricultural Sciences
3,Involving citizens in monitoring the Kunming–M...,https://www.nature.com/articles/s41893-024-014...,"Finn Danielsen, Natasha Ali, Neil D. Burgess",2024-10-18,https://www.nature.com/articles/s41893-024-014...,Open Access,Agricultural Sciences
4,A renewable and socially accepted energy syste...,https://www.nature.com/articles/s41893-024-014...,"Guillermo Valenzuela-Venegas, Maria Luisa Lode...",2024-10-11,https://www.nature.com/articles/s41893-024-014...,,Agricultural Sciences
...,...,...,...,...,...,...,...
729,Water competition between cities and agricultu...,https://www.nature.com/articles/s41893-017-0006-8,"Martina Flörke, Christof Schneider, Robert I. ...",2018-01-08,https://www.nature.com/articles/s41893-017-000...,,Agricultural Sciences
730,Impacts of booming concrete production on wate...,https://www.nature.com/articles/s41893-017-0009-5,"Sabbie A. Miller, Arpad Horvath, Paulo J. M. M...",2018-01-08,https://www.nature.com/articles/s41893-017-000...,,Agricultural Sciences
731,Targeted emission reductions from global super...,https://www.nature.com/articles/s41893-017-0003-y,"Dan Tong, Qiang Zhang, Kebin He",2018-01-08,https://www.nature.com/articles/s41893-017-000...,,Agricultural Sciences
732,Increased vegetation growth and carbon stock i...,https://www.nature.com/articles/s41893-017-0004-x,"Xiaowei Tong, Martin Brandt, Rasmus Fensholt",2018-01-08,https://www.nature.com/articles/s41893-017-000...,,Agricultural Sciences


#### Selecting the 'Open Access' articles published before the advent of ChatGPT

ChatGPT was released by OpenAI on November 30, 2022.

In [6]:
df_nature_sustainability_open_access = df_nature_sustainability[df_nature_sustainability['Open Access'] == 'Open Access']
df_nature_sustainability_open_access = df_nature_sustainability_open_access[df_nature_sustainability_open_access['Published'] <= '2022-11-30']
df_nature_sustainability_open_access = df_nature_sustainability_open_access.reset_index(drop=True)
df_nature_sustainability_open_access

,Title,URL,Authors (compact list),Published,PDF URL,Open Access,Area of Knowledge
0,Towards process-oriented management of tropica...,https://www.nature.com/articles/s41893-022-009...,"Raphael Seguin, David Mouillot, Nicolas Loiseau",2022-11-14,https://www.nature.com/articles/s41893-022-009...,Open Access,Agricultural Sciences
1,Impacts of meeting minimum access on critical ...,https://www.nature.com/articles/s41893-022-009...,"Crelis F. Rammelt, Joyeeta Gupta, Caroline Zimm",2022-11-10,https://www.nature.com/articles/s41893-022-009...,Open Access,Agricultural Sciences
2,Sustained productivity and agronomic potential...,https://www.nature.com/articles/s41893-022-009...,"Shilai Zhang, Guangfu Huang, Fengyi Hu",2022-11-07,https://www.nature.com/articles/s41893-022-009...,Open Access,Agricultural Sciences
3,Protected area personnel and ranger numbers ar...,https://www.nature.com/articles/s41893-022-009...,"Michael R. Appleton, Alexandre Courtiol, Barne...",2022-10-20,https://www.nature.com/articles/s41893-022-009...,Open Access,Agricultural Sciences
4,Insect pest management with sex pheromone prec...,https://www.nature.com/articles/s41893-022-009...,"Hong-Lei Wang, Bao-Jian Ding, Christer Löfstedt",2022-09-01,https://www.nature.com/articles/s41893-022-009...,Open Access,Agricultural Sciences
5,Quantifying Earth system interactions for sust...,https://www.nature.com/articles/s41893-022-009...,"Anna Chrysafi, Vili Virkki, Matti Kummu",2022-08-15,https://www.nature.com/articles/s41893-022-009...,Open Access,Agricultural Sciences
6,Persistent impact of Fukushima decontamination...,https://www.nature.com/articles/s41893-022-009...,"Bin Feng, Yuichi Onda, Yupan Zhang",2022-07-14,https://www.nature.com/articles/s41893-022-009...,Open Access,Agricultural Sciences
7,Scientific evidence on the political impact of...,https://www.nature.com/articles/s41893-022-009...,"Frank Biermann, Thomas Hickmann, Birka Wicke",2022-06-20,https://www.nature.com/articles/s41893-022-009...,Open Access,Agricultural Sciences
8,Re-thinking procurement incentives for electri...,https://www.nature.com/articles/s41893-022-008...,"Ashley Nunes, Lucas Woodley, Philip Rossetti",2022-04-04,https://www.nature.com/articles/s41893-022-008...,Open Access,Agricultural Sciences
9,Improving biodiversity protection through arti...,https://www.nature.com/articles/s41893-022-008...,"Daniele Silvestro, Stefano Goria, Alexandre An...",2022-03-24,https://www.nature.com/articles/s41893-022-008...,Open Access,Agricultural Sciences


#### Inspecting a few articles

In [7]:
df_nature_sustainability_open_access.at[0, 'PDF URL']

'https://www.nature.com/articles/s41893-022-00981-x.pdf'

#### Exporting to a file

In [9]:
df_nature_sustainability_open_access.to_json(f"{output_directory}/nature_sustainability_open_access.jsonl", orient='records', lines=True)

### Scraping [Plant Biotechnology Journal](https://onlinelibrary.wiley.com/journal/14677652?msockid=0525cb73d9a76a060b80df20d87e6b4b)

## Applied Social Sciences

### Scraping [Journal of Applied Social Science](https://journals.sagepub.com/home/jax)

### Scraping [Journal of Social Issues](https://spssi.onlinelibrary.wiley.com/journal/15404560)

### Scraping [Social Science & Medicine](https://www.sciencedirect.com/journal/social-science-and-medicine)

### Scraping [Sociological Practice Association](https://digitalcommons.wayne.edu/socprac/)

## Biological Sciences

### Scraping [Cell](https://www.cell.com/cell/home)

### Scraping [Nature Medicine](https://www.nature.com/nm/)

#### Scraping the articles

In [10]:
root_url = 'https://www.nature.com'
article_list_url = 'https://www.nature.com/nm/research-articles?searchType=journalSearch&sort=PubDate&page='
start_page = 1
end_page = 241
area_of_knowledge = 'Biological Sciences'

In [11]:
# Initialize an empty list to store the data
data = []

# Iterating through the URLs and using 'tqdm' for progress tracking in the range loop
for i in tqdm(range(start_page, end_page + 1)):
    url = f"{article_list_url}{i}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')

    # Finding all <article> elements with the class 'u-full-height c-card c-card--flush'
    articles = soup.find_all('article', class_='u-full-height c-card c-card--flush')

    for article in articles:
        # Extracting the title
        title_tag = article.find('h3', class_='c-card__title').find('a')
        title = title_tag.get_text(strip=True) if title_tag else ''
        title_url = f"{root_url}{title_tag.get('href')}" if title_tag else ''

        # Extracting the authors
        author_tags = article.find_all('li', itemprop='creator')
        authors = ', '.join(author.get_text(strip=True) for author in author_tags)

        # Extracting the published date
        date_tag = article.find('time')
        date_published = date_tag['datetime'] if date_tag else ''

        # Extracting the PDF URL
        pdf_url = f"{title_url}.pdf" if title_url else ''

        # Extracting the 'Open Access' label
        open_access_tag = article.find('span', class_='u-color-open-access')
        open_access = open_access_tag.get_text(strip=True) if open_access_tag else ''

        # Appending the data to the list
        data.append({
            'Title': title,
            'URL': title_url,
            'Authors (compact list)': authors,
            'Published': date_published,
            'PDF URL': pdf_url,
            'Open Access': open_access,
            'Area of Knowledge': area_of_knowledge
        })

# Creating a DataFrame from the data
df_nature_medicine = pd.DataFrame(data)

100%|██████████| 241/241 [06:31<00:00,  1.62s/it]


In [12]:
df_nature_medicine['Published'] = pd.to_datetime(df_nature_medicine['Published'])
df_nature_medicine

,Title,URL,Authors (compact list),Published,PDF URL,Open Access,Area of Knowledge
0,Semaglutide in patients with overweight or obe...,https://www.nature.com/articles/s41591-024-033...,"Ellen M. Apperloo, Jose L. Gorriz, Hiddo J. L....",2024-10-25,https://www.nature.com/articles/s41591-024-033...,,Biological Sciences
1,Subclassification of obesity for precision pre...,https://www.nature.com/articles/s41591-024-032...,"Daniel E. Coral, Femke Smit, Paul W. Franks",2024-10-24,https://www.nature.com/articles/s41591-024-032...,Open Access,Biological Sciences
2,Global genomic surveillance of monkeypox virus,https://www.nature.com/articles/s41591-024-033...,"James R. Otieno, Christopher Ruis, Lorenzo Sub...",2024-10-23,https://www.nature.com/articles/s41591-024-033...,,Biological Sciences
3,Home-based transcranial direct current stimula...,https://www.nature.com/articles/s41591-024-033...,"Rachel D. Woodham, Sudhakar Selvaraj, Cynthia ...",2024-10-21,https://www.nature.com/articles/s41591-024-033...,Open Access,Biological Sciences
4,AAV gene therapy for Duchenne muscular dystrop...,https://www.nature.com/articles/s41591-024-033...,"Jerry R. Mendell, Francesco Muntoni, Louise R....",2024-10-09,https://www.nature.com/articles/s41591-024-033...,Open Access,Biological Sciences
...,...,...,...,...,...,...,...
4804,Microsatellite instability in primary neoplasm...,https://www.nature.com/articles/nm0195-65,"Gauri C. Bedi, William H. Westra, David Sidransky",1995-01-01,https://www.nature.com/articles/nm0195-65.pdf,,Biological Sciences
4805,Serial magnetic resonance imaging of experimen...,https://www.nature.com/articles/nm0195-69,"Michael P. Skinner, Chun Yuan, Russell Ross",1995-01-01,https://www.nature.com/articles/nm0195-69.pdf,,Biological Sciences
4806,Liposome-mediatedCFTRgene transfer to the nasa...,https://www.nature.com/articles/nm0195-39,"Natasha J. Caplen, Eric W.F.W. Alton, Duncan M...",1995-01-01,https://www.nature.com/articles/nm0195-39.pdf,,Biological Sciences
4807,"The paternal inheritance of the centrosome, th...",https://www.nature.com/articles/nm0195-47,"Calvin Simerly, Gwo-Jang Wu, Gerald Schatten",1995-01-01,https://www.nature.com/articles/nm0195-47.pdf,,Biological Sciences


#### Selecting the 'Open Access' articles published before the advent of ChatGPT

ChatGPT was released by OpenAI on November 30, 2022.

In [13]:
df_nature_medicine_open_access = df_nature_medicine[df_nature_medicine['Open Access'] == 'Open Access']
df_nature_medicine_open_access = df_nature_medicine_open_access[df_nature_medicine_open_access['Published'] <= '2022-11-30']
df_nature_medicine_open_access = df_nature_medicine_open_access.reset_index(drop=True)
df_nature_medicine_open_access

,Title,URL,Authors (compact list),Published,PDF URL,Open Access,Area of Knowledge
0,CSF tau microtubule-binding region identifies ...,https://www.nature.com/articles/s41591-022-020...,"Kanta Horie, Nicolas R. Barthélemy, Chihiro Sato",2022-11-24,https://www.nature.com/articles/s41591-022-020...,Open Access,Biological Sciences
1,Global and regional estimates of orphans attri...,https://www.nature.com/articles/s41591-022-021...,"Florence Guida, Rachel Kidman, Valerie McCormack",2022-11-20,https://www.nature.com/articles/s41591-022-021...,Open Access,Biological Sciences
2,Amyloid and tau PET-positive cognitively unimp...,https://www.nature.com/articles/s41591-022-020...,"Rik Ossenkoppele, Alexa Pichet Binette, Oskar ...",2022-11-10,https://www.nature.com/articles/s41591-022-020...,Open Access,Biological Sciences
3,Acute and postacute sequelae associated with S...,https://www.nature.com/articles/s41591-022-020...,"Benjamin Bowe, Yan Xie, Ziyad Al-Aly",2022-11-10,https://www.nature.com/articles/s41591-022-020...,Open Access,Biological Sciences
4,Rare and common genetic determinants of metabo...,https://www.nature.com/articles/s41591-022-020...,"Praveen Surendran, Isobel D. Stewart, Claudia ...",2022-11-10,https://www.nature.com/articles/s41591-022-020...,Open Access,Biological Sciences
...,...,...,...,...,...,...,...
171,Investigation of the role of typhoid toxin in ...,https://www.nature.com/articles/s41591-019-0505-4,"Malick M. Gibani, Elizabeth Jones, Andrew J. P...",2019-07-03,https://www.nature.com/articles/s41591-019-050...,Open Access,Biological Sciences
172,The vaginal microbiome and preterm birth,https://www.nature.com/articles/s41591-019-0450-2,"Jennifer M. Fettweis, Myrna G. Serrano, Gregor...",2019-05-29,https://www.nature.com/articles/s41591-019-045...,Open Access,Biological Sciences
173,IFN-γ-independent immune markers ofMycobacteri...,https://www.nature.com/articles/s41591-019-0441-3,"Lenette L. Lu, Malisa T. Smith, Galit Alter",2019-05-20,https://www.nature.com/articles/s41591-019-044...,Open Access,Biological Sciences
174,A patient-level pooled analysis of treatment-s...,https://www.nature.com/articles/s41591-018-0224-2,"Marjorie Z. Imperial, Payam Nahid, Rada M. Savic",2018-11-05,https://www.nature.com/articles/s41591-018-022...,Open Access,Biological Sciences


#### Inspecting a few articles

In [14]:
df_nature_medicine_open_access.at[0, 'PDF URL']

'https://www.nature.com/articles/s41591-022-02075-9.pdf'

#### Exporting to a file

In [16]:
df_nature_medicine_open_access.to_json(f"{output_directory}/nature_medicine_open_access.jsonl", orient='records', lines=True)

### Scraping [Nature Reviews Molecular Cell Biology](https://www.nature.com/nrm/)

This journal had no 'Open Access' articles at the time of data collection.

## Engineering

### Scraping [Advanced Materials](https://onlinelibrary.wiley.com/journal/15214095?msockid=0525cb73d9a76a060b80df20d87e6b4b)

### Scraping [IEEE Communications Surveys and Tutorials](https://ieeexplore.ieee.org/document/9621329)

### Scraping [IEEE Transactions on Robotics](https://ieeexplore.ieee.org/xpl/RecentIssue.jsp?punumber=8860)

### Scraping [Nature Materials](https://www.nature.com/nmat/)

#### Scraping the articles

In [17]:
root_url = 'https://www.nature.com'
article_list_url = 'https://www.nature.com/nmat/research-articles?searchType=journalSearch&sort=PubDate&page='
start_page = 1
end_page = 156
area_of_knowledge = 'Engineering'

In [18]:
# Initialize an empty list to store the data
data = []

# Iterating through the URLs and using 'tqdm' for progress tracking in the range loop
for i in tqdm(range(start_page, end_page + 1)):
    url = f"{article_list_url}{i}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')

    # Finding all <article> elements with the class 'u-full-height c-card c-card--flush'
    articles = soup.find_all('article', class_='u-full-height c-card c-card--flush')

    for article in articles:
        # Extracting the title
        title_tag = article.find('h3', class_='c-card__title').find('a')
        title = title_tag.get_text(strip=True) if title_tag else ''
        title_url = f"{root_url}{title_tag.get('href')}" if title_tag else ''

        # Extracting the authors
        author_tags = article.find_all('li', itemprop='creator')
        authors = ', '.join(author.get_text(strip=True) for author in author_tags)

        # Extracting the published date
        date_tag = article.find('time')
        date_published = date_tag['datetime'] if date_tag else ''

        # Extracting the PDF URL
        pdf_url = f"{title_url}.pdf" if title_url else ''

        # Extracting the 'Open Access' label
        open_access_tag = article.find('span', class_='u-color-open-access')
        open_access = open_access_tag.get_text(strip=True) if open_access_tag else ''

        # Appending the data to the list
        data.append({
            'Title': title,
            'URL': title_url,
            'Authors (compact list)': authors,
            'Published': date_published,
            'PDF URL': pdf_url,
            'Open Access': open_access,
            'Area of Knowledge': area_of_knowledge
        })

# Creating a DataFrame from the data
df_nature_materials = pd.DataFrame(data)

100%|██████████| 156/156 [04:08<00:00,  1.60s/it]


In [19]:
df_nature_materials['Published'] = pd.to_datetime(df_nature_materials['Published'])
df_nature_materials

,Title,URL,Authors (compact list),Published,PDF URL,Open Access,Area of Knowledge
0,Spin switching in Sm0.7Er0.3FeO3triggered by t...,https://www.nature.com/articles/s41563-024-020...,"Zhenya Zhang, Minoru Kanega, Hideki Hirori",2024-10-25,https://www.nature.com/articles/s41563-024-020...,,Engineering
1,"Formation, chemical evolution and solidificati...",https://www.nature.com/articles/s41563-024-020...,"Biao Jin, Ying Chen, James J. De Yoreo",2024-10-24,https://www.nature.com/articles/s41563-024-020...,,Engineering
2,Antagonistic-contracting high-power photo-osci...,https://www.nature.com/articles/s41563-024-020...,"Yusen Zhao, Zixiao Liu, Ximin He",2024-10-24,https://www.nature.com/articles/s41563-024-020...,,Engineering
3,Obtaining V2(PO4)3by sodium extraction from si...,https://www.nature.com/articles/s41563-024-020...,"Sunkyu Park, Ziliang Wang, Christian Masquelier",2024-10-23,https://www.nature.com/articles/s41563-024-020...,,Engineering
4,Coherent optical spin Hall transport for polar...,https://www.nature.com/articles/s41563-024-020...,"Ying Shi, Yusong Gan, Qihua Xiong",2024-10-22,https://www.nature.com/articles/s41563-024-020...,,Engineering
...,...,...,...,...,...,...,...
3101,Bridge-mediated hopping or superexchange elect...,https://www.nature.com/articles/nmat706,"Christoph Lambert, Gilbert Nöll, Jürgen Schelter",2002-09-02,https://www.nature.com/articles/nmat706.pdf,,Engineering
3102,Polymer-stabilized liquid crystal blue phases,https://www.nature.com/articles/nmat712,"Hirotsugu Kikuchi, Masayuki Yokota, Tisato Kaj...",2002-09-02,https://www.nature.com/articles/nmat712.pdf,,Engineering
3103,Closed-loop phase behaviour in block copolymers,https://www.nature.com/articles/nmat724,"Du Yeol Ryu, Unyong Jeong, Thomas P. Russell",2002-09-02,https://www.nature.com/articles/nmat724.pdf,,Engineering
3104,In situepitaxial MgB2thin films for supercondu...,https://www.nature.com/articles/nmat703,"Xianghui Zeng, Alexej V. Pogrebnyakov, Zi-Kui Liu",2002-09-02,https://www.nature.com/articles/nmat703.pdf,,Engineering


#### Selecting the 'Open Access' articles published before the advent of ChatGPT

ChatGPT was released by OpenAI on November 30, 2022.

In [20]:
df_nature_materials_open_access = df_nature_materials[df_nature_materials['Open Access'] == 'Open Access']
df_nature_materials_open_access = df_nature_materials_open_access[df_nature_materials_open_access['Published'] <= '2022-11-30']
df_nature_materials_open_access = df_nature_materials_open_access.reset_index(drop=True)
df_nature_materials_open_access

,Title,URL,Authors (compact list),Published,PDF URL,Open Access,Area of Knowledge
0,Exchange controlled triplet fusion in metal–or...,https://www.nature.com/articles/s41563-022-013...,"Dong-Gwang Ha, Ruomeng Wan, Mircea Dincă",2022-10-06,https://www.nature.com/articles/s41563-022-013...,Open Access,Engineering
1,Robust cholesteric liquid crystal elastomer fi...,https://www.nature.com/articles/s41563-022-013...,"Yong Geng, Rijeesh Kizhakidathazhath, Jan P. F...",2022-09-29,https://www.nature.com/articles/s41563-022-013...,Open Access,Engineering
2,Giant voltage amplification from electrostatic...,https://www.nature.com/articles/s41563-022-013...,"Mónica Graf, Hugo Aramberri, Jorge Íñiguez",2022-08-25,https://www.nature.com/articles/s41563-022-013...,Open Access,Engineering
3,Cell clusters softening triggers collective ce...,https://www.nature.com/articles/s41563-022-013...,"Cristian L. Marchant, Abdul N. Malmi-Kakkada, ...",2022-08-15,https://www.nature.com/articles/s41563-022-013...,Open Access,Engineering
4,Unconventional excitonic states with phonon si...,https://www.nature.com/articles/s41563-022-012...,"Ling Zhou, Junwei Huang, Hongtao Yuan",2022-06-16,https://www.nature.com/articles/s41563-022-012...,Open Access,Engineering
5,Programmable gear-based mechanical metamaterials,https://www.nature.com/articles/s41563-022-012...,"Xin Fang, Jihong Wen, Peter Gumbsch",2022-06-09,https://www.nature.com/articles/s41563-022-012...,Open Access,Engineering
6,3D printed protein-based robotic structures ac...,https://www.nature.com/articles/s41563-022-012...,"Haiyang Jia, Johannes Flommersfeld, Petra Schw...",2022-05-26,https://www.nature.com/articles/s41563-022-012...,Open Access,Engineering
7,STING agonist delivery by tumour-penetrating P...,https://www.nature.com/articles/s41563-022-012...,"Eric L. Dane, Alexis Belessiotis-Richards, Dar...",2022-05-23,https://www.nature.com/articles/s41563-022-012...,Open Access,Engineering
8,Trace removal of benzene vapour using double-w...,https://www.nature.com/articles/s41563-022-012...,"Tao He, Xiang-Jing Kong, Jian-Rong Li",2022-04-28,https://www.nature.com/articles/s41563-022-012...,Open Access,Engineering
9,"Superconductivity in (Ba,K)SbO3",https://www.nature.com/articles/s41563-022-012...,"Minu Kim, Graham M. McNally, Hidenori Takagi",2022-02-28,https://www.nature.com/articles/s41563-022-012...,Open Access,Engineering


#### Inspecting a few articles

In [21]:
df_nature_materials_open_access.at[0, 'PDF URL']

'https://www.nature.com/articles/s41563-022-01368-1.pdf'

#### Exporting to a file

In [22]:
df_nature_materials_open_access.to_json(f"{output_directory}/nature_materials_open_access.jsonl", orient='records', lines=True)

### Scraping [Nature Nanotechnology](https://www.nature.com/nnano/)

#### Scraping the articles

In [23]:
root_url = 'https://www.nature.com'
article_list_url = 'https://www.nature.com/nnano/research-articles?searchType=journalSearch&sort=PubDate&page='
start_page = 1
end_page = 115
area_of_knowledge = 'Engineering'

In [24]:
# Initialize an empty list to store the data
data = []

# Iterating through the URLs and using 'tqdm' for progress tracking in the range loop
for i in tqdm(range(start_page, end_page + 1)):
    url = f"{article_list_url}{i}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')

    # Finding all <article> elements with the class 'u-full-height c-card c-card--flush'
    articles = soup.find_all('article', class_='u-full-height c-card c-card--flush')

    for article in articles:
        # Extracting the title
        title_tag = article.find('h3', class_='c-card__title').find('a')
        title = title_tag.get_text(strip=True) if title_tag else ''
        title_url = f"{root_url}{title_tag.get('href')}" if title_tag else ''

        # Extracting the authors
        author_tags = article.find_all('li', itemprop='creator')
        authors = ', '.join(author.get_text(strip=True) for author in author_tags)

        # Extracting the published date
        date_tag = article.find('time')
        date_published = date_tag['datetime'] if date_tag else ''

        # Extracting the PDF URL
        pdf_url = f"{title_url}.pdf" if title_url else ''

        # Extracting the 'Open Access' label
        open_access_tag = article.find('span', class_='u-color-open-access')
        open_access = open_access_tag.get_text(strip=True) if open_access_tag else ''

        # Appending the data to the list
        data.append({
            'Title': title,
            'URL': title_url,
            'Authors (compact list)': authors,
            'Published': date_published,
            'PDF URL': pdf_url,
            'Open Access': open_access,
            'Area of Knowledge': area_of_knowledge
        })

# Creating a DataFrame from the data
df_nature_nanotechnology = pd.DataFrame(data)

100%|██████████| 115/115 [03:01<00:00,  1.57s/it]


In [25]:
df_nature_nanotechnology['Published'] = pd.to_datetime(df_nature_nanotechnology['Published'])
df_nature_nanotechnology

,Title,URL,Authors (compact list),Published,PDF URL,Open Access,Area of Knowledge
0,Synthetic syntrophy for adenine nucleotide cro...,https://www.nature.com/articles/s41565-024-018...,"Laura Heinen, Marco van den Noort, Bert Poolman",2024-10-21,https://www.nature.com/articles/s41565-024-018...,Open Access,Engineering
1,Out-of-plane coordination of iridium single at...,https://www.nature.com/articles/s41565-024-018...,"Jie Zhao, Yue Guo, Francesco Ciucci",2024-10-21,https://www.nature.com/articles/s41565-024-018...,Open Access,Engineering
2,Linearly programmable two-dimensional halide p...,https://www.nature.com/articles/s41565-024-017...,"Seung Ju Kim, In Hyuk Im, Ho Won Jang",2024-10-18,https://www.nature.com/articles/s41565-024-017...,,Engineering
3,Magnetoelectric nanodiscs enable wireless tran...,https://www.nature.com/articles/s41565-024-017...,"Ye Ji Kim, Noah Kent, Polina Anikeeva",2024-10-11,https://www.nature.com/articles/s41565-024-017...,Open Access,Engineering
4,Rapid precision targeting of nanoparticles to ...,https://www.nature.com/articles/s41565-024-017...,"Tapas R. Nayak, Adrian Chrastina, Jan E. Schni...",2024-10-08,https://www.nature.com/articles/s41565-024-017...,,Engineering
...,...,...,...,...,...,...,...
2280,A nanoplasmonic molecular ruler for measuring ...,https://www.nature.com/articles/nnano.2006.51,"Gang L. Liu, Yadong Yin, Fanqing Frank Chen",2006-10-04,https://www.nature.com/articles/nnano.2006.51.pdf,,Engineering
2281,Giant piezoresistance effect in silicon nanowires,https://www.nature.com/articles/nnano.2006.53,"Rongrui He, Peidong Yang",2006-10-04,https://www.nature.com/articles/nnano.2006.53.pdf,,Engineering
2282,Digital memory device based on tobacco mosaic ...,https://www.nature.com/articles/nnano.2006.55,"Ricky J. Tseng, Chunglin Tsai, Yang Yang",2006-10-04,https://www.nature.com/articles/nnano.2006.55.pdf,,Engineering
2283,Torsional electromechanical quantum oscillatio...,https://www.nature.com/articles/nnano.2006.57,"Tzahi Cohen-Karni, Lior Segev, Ernesto Joselevich",2006-10-04,https://www.nature.com/articles/nnano.2006.57.pdf,,Engineering


#### Selecting the 'Open Access' articles published before the advent of ChatGPT

ChatGPT was released by OpenAI on November 30, 2022.

In [26]:
df_nature_nanotechnology_open_access = df_nature_nanotechnology[df_nature_nanotechnology['Open Access'] == 'Open Access']
df_nature_nanotechnology_open_access = df_nature_nanotechnology_open_access[df_nature_nanotechnology_open_access['Published'] <= '2022-11-30']
df_nature_nanotechnology_open_access = df_nature_nanotechnology_open_access.reset_index(drop=True)
df_nature_nanotechnology_open_access

,Title,URL,Authors (compact list),Published,PDF URL,Open Access,Area of Knowledge
0,Potential use of engineered nanoparticles in o...,https://www.nature.com/articles/s41565-022-012...,"Peyman Babakhani, Tanapon Phenrat, Michael F. ...",2022-11-28,https://www.nature.com/articles/s41565-022-012...,Open Access,Engineering
1,Feedback-controlled hydrogels with homeostatic...,https://www.nature.com/articles/s41565-022-012...,"Hang Zhang, Hao Zeng, Olli Ikkala",2022-11-28,https://www.nature.com/articles/s41565-022-012...,Open Access,Engineering
2,Quantum Hall phase in graphene engineered by i...,https://www.nature.com/articles/s41565-022-012...,"Yaning Wang, Xiang Gao, Zhidong Zhang",2022-11-21,https://www.nature.com/articles/s41565-022-012...,Open Access,Engineering
3,Hierarchically self-assembled homochiral helic...,https://www.nature.com/articles/s41565-022-012...,"Cong Du, Zujian Li, Minghua Liu",2022-11-03,https://www.nature.com/articles/s41565-022-012...,Open Access,Engineering
4,Human-muscle-inspired single fibre actuator wi...,https://www.nature.com/articles/s41565-022-012...,"In Ho Kim, Subi Choi, Sang Ouk Kim",2022-10-27,https://www.nature.com/articles/s41565-022-012...,Open Access,Engineering
5,Nano-optical designs for high-efficiency monol...,https://www.nature.com/articles/s41565-022-012...,"Philipp Tockhorn, Johannes Sutter, Christiane ...",2022-10-24,https://www.nature.com/articles/s41565-022-012...,Open Access,Engineering
6,Local and global energy barriers for chiral do...,https://www.nature.com/articles/s41565-022-012...,"Jiho Yoon, See-Hun Yang, Stuart. S. P. Parkin",2022-10-06,https://www.nature.com/articles/s41565-022-012...,Open Access,Engineering
7,A single hole spin with enhanced coherence in ...,https://www.nature.com/articles/s41565-022-011...,"N. Piot, B. Brun, S. De Franceschi",2022-09-22,https://www.nature.com/articles/s41565-022-011...,Open Access,Engineering
8,Three-dimensional racetrack memory devices des...,https://www.nature.com/articles/s41565-022-012...,"Ke Gu, Yicheng Guan, Stuart S. P. Parkin",2022-09-22,https://www.nature.com/articles/s41565-022-012...,Open Access,Engineering
9,Doping-driven topological polaritons in graphe...,https://www.nature.com/articles/s41565-022-011...,"Hai Hu, Na Chen, Qing Dai",2022-08-18,https://www.nature.com/articles/s41565-022-011...,Open Access,Engineering


#### Inspecting a few articles

In [27]:
df_nature_nanotechnology_open_access.at[0, 'PDF URL']

'https://www.nature.com/articles/s41565-022-01226-w.pdf'

#### Exporting to a file

In [28]:
df_nature_nanotechnology_open_access.to_json(f"{output_directory}/nature_nanotechnology_open_access.jsonl", orient='records', lines=True)

## Exact and Earth Sciences

### Scraping [Annual Review of Earth and Planetary Sciences](https://www.annualreviews.org/content/journals/earth)

### Scraping [Journal of Geophysical Research: Solid Earth](https://agupubs.onlinelibrary.wiley.com/journal/21699356)

### Scraping [Nature Geoscience](https://www.nature.com/ngeo/)

#### Scraping the articles

In [29]:
root_url = 'https://www.nature.com'
article_list_url = 'https://www.nature.com/ngeo/research-articles?searchType=journalSearch&sort=PubDate&page='
start_page = 1
end_page = 115
area_of_knowledge = 'Exact and Earth Sciences'

In [30]:
# Initialize an empty list to store the data
data = []

# Iterating through the URLs and using 'tqdm' for progress tracking in the range loop
for i in tqdm(range(start_page, end_page + 1)):
    url = f"{article_list_url}{i}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')

    # Finding all <article> elements with the class 'u-full-height c-card c-card--flush'
    articles = soup.find_all('article', class_='u-full-height c-card c-card--flush')

    for article in articles:
        # Extracting the title
        title_tag = article.find('h3', class_='c-card__title').find('a')
        title = title_tag.get_text(strip=True) if title_tag else ''
        title_url = f"{root_url}{title_tag.get('href')}" if title_tag else ''

        # Extracting the authors
        author_tags = article.find_all('li', itemprop='creator')
        authors = ', '.join(author.get_text(strip=True) for author in author_tags)

        # Extracting the published date
        date_tag = article.find('time')
        date_published = date_tag['datetime'] if date_tag else ''

        # Extracting the PDF URL
        pdf_url = f"{title_url}.pdf" if title_url else ''

        # Extracting the 'Open Access' label
        open_access_tag = article.find('span', class_='u-color-open-access')
        open_access = open_access_tag.get_text(strip=True) if open_access_tag else ''

        # Appending the data to the list
        data.append({
            'Title': title,
            'URL': title_url,
            'Authors (compact list)': authors,
            'Published': date_published,
            'PDF URL': pdf_url,
            'Open Access': open_access,
            'Area of Knowledge': area_of_knowledge
        })

# Creating a DataFrame from the data
df_nature_geoscience = pd.DataFrame(data)

100%|██████████| 115/115 [03:04<00:00,  1.61s/it]


In [31]:
df_nature_geoscience['Published'] = pd.to_datetime(df_nature_geoscience['Published'])
df_nature_geoscience

,Title,URL,Authors (compact list),Published,PDF URL,Open Access,Area of Knowledge
0,Enhanced ocean CO2uptake due to near-surface t...,https://www.nature.com/articles/s41561-024-015...,"Daniel J. Ford, Jamie D. Shutler, Ian Ashton",2024-10-25,https://www.nature.com/articles/s41561-024-015...,Open Access,Exact and Earth Sciences
1,Direct and lagged climate change effects inten...,https://www.nature.com/articles/s41561-024-015...,"Emanuele Bevacqua, Oldrich Rakovec, Jakob Zsch...",2024-10-21,https://www.nature.com/articles/s41561-024-015...,Open Access,Exact and Earth Sciences
2,Large-scale mantle heterogeneity as a legacy o...,https://www.nature.com/articles/s41561-024-015...,"Luc S. Doucet, Zheng-Xiang Li",2024-10-18,https://www.nature.com/articles/s41561-024-015...,Open Access,Exact and Earth Sciences
3,Centennial-scale variations in the carbon cycl...,https://www.nature.com/articles/s41561-024-015...,"Etienne Legrain, Emilie Capron, Thomas F. Stocker",2024-10-11,https://www.nature.com/articles/s41561-024-015...,,Exact and Earth Sciences
4,Nitrogen fixation in the North Atlantic suppor...,https://www.nature.com/articles/s41561-024-015...,"Cora Hoerstmann, Borja Aguiar-González, Mar Be...",2024-10-11,https://www.nature.com/articles/s41561-024-015...,Open Access,Exact and Earth Sciences
...,...,...,...,...,...,...,...
2284,High rates of sea-level rise during the last i...,https://www.nature.com/articles/ngeo.2007.28,"E. J. Rohling, K. Grant, M. Kucera",2007-12-16,https://www.nature.com/articles/ngeo.2007.28.pdf,,Exact and Earth Sciences
2285,Persistent earthquake clusters and gaps from s...,https://www.nature.com/articles/ngeo.2007.36,Tom Parsons,2007-12-09,https://www.nature.com/articles/ngeo.2007.36.pdf,,Exact and Earth Sciences
2286,Atmospheric carbon dioxide linked with Mesozoi...,https://www.nature.com/articles/ngeo.2007.29,"Benjamin J. Fletcher, Stuart J. Brentnall, Dav...",2007-12-09,https://www.nature.com/articles/ngeo.2007.29.pdf,,Exact and Earth Sciences
2287,Influence of brine formation on Arctic Ocean c...,https://www.nature.com/articles/ngeo.2007.5,"Brian A. Haley, Martin Frank, Anton Eisenhauer",2007-12-02,https://www.nature.com/articles/ngeo.2007.5.pdf,,Exact and Earth Sciences


#### Selecting the 'Open Access' articles published before the advent of ChatGPT

ChatGPT was released by OpenAI on November 30, 2022.

In [32]:
df_nature_geoscience_open_access = df_nature_geoscience[df_nature_geoscience['Open Access'] == 'Open Access']
df_nature_geoscience_open_access = df_nature_geoscience_open_access[df_nature_geoscience_open_access['Published'] <= '2022-11-30']
df_nature_geoscience_open_access = df_nature_geoscience_open_access.reset_index(drop=True)
df_nature_geoscience_open_access

,Title,URL,Authors (compact list),Published,PDF URL,Open Access,Area of Knowledge
0,Limited carbon cycling due to high-pressure ef...,https://www.nature.com/articles/s41561-022-010...,"Chie Amano, Zihao Zhao, Gerhard J. Herndl",2022-11-28,https://www.nature.com/articles/s41561-022-010...,Open Access,Exact and Earth Sciences
1,Global patterns in marine organic matter stoic...,https://www.nature.com/articles/s41561-022-010...,"Keisuke Inomura, Curtis Deutsch, Michael J. Fo...",2022-11-21,https://www.nature.com/articles/s41561-022-010...,Open Access,Exact and Earth Sciences
2,Carbon dioxide sink in the Arctic Ocean from c...,https://www.nature.com/articles/s41561-022-010...,"Andreas Rogge, Markus Janout, Anya M. Waite",2022-11-21,https://www.nature.com/articles/s41561-022-010...,Open Access,Exact and Earth Sciences
3,Fraser Island (K'gari) and initiation of the G...,https://www.nature.com/articles/s41561-022-010...,"D. Ellerton, T. M. Rittenour, X. Zhao",2022-11-14,https://www.nature.com/articles/s41561-022-010...,Open Access,Exact and Earth Sciences
4,Declining metal availability in the Mesozoic s...,https://www.nature.com/articles/s41561-022-010...,"Qiong Zhang, El Mahdi Bendif, Rosalind E. M. R...",2022-10-31,https://www.nature.com/articles/s41561-022-010...,Open Access,Exact and Earth Sciences
5,Millennial-scale climate variability over land...,https://www.nature.com/articles/s41561-022-010...,"R. Hébert, U. Herzschuh, T. Laepple",2022-10-31,https://www.nature.com/articles/s41561-022-010...,Open Access,Exact and Earth Sciences
6,Long-range predictability of extratropical cli...,https://www.nature.com/articles/s41561-022-010...,"A. A. Scaife, L. Hermanson, D. Smith",2022-10-03,https://www.nature.com/articles/s41561-022-010...,Open Access,Exact and Earth Sciences
7,Submarine melting of glaciers in Greenland amp...,https://www.nature.com/articles/s41561-022-010...,"D. A. Slater, F. Straneo",2022-10-03,https://www.nature.com/articles/s41561-022-010...,Open Access,Exact and Earth Sciences
8,Carbon and sediment fluxes inhibited in the su...,https://www.nature.com/articles/s41561-022-010...,"Ed L. Pope, Maarten S. Heijnen, Morelia Urlaub",2022-09-29,https://www.nature.com/articles/s41561-022-010...,Open Access,Exact and Earth Sciences
9,Pleistocene climate variability in eastern Afr...,https://www.nature.com/articles/s41561-022-010...,"Verena Foerster, Asfawossen Asrat, Martin H. T...",2022-09-26,https://www.nature.com/articles/s41561-022-010...,Open Access,Exact and Earth Sciences


#### Inspecting a few articles

In [33]:
df_nature_geoscience_open_access.at[0, 'PDF URL']

'https://www.nature.com/articles/s41561-022-01081-3.pdf'

#### Exporting to a file

In [34]:
df_nature_geoscience_open_access.to_json(f"{output_directory}/nature_geoscience_open_access.jsonl", orient='records', lines=True)

### Scraping [Reviews of Geophysics](https://agupubs.onlinelibrary.wiley.com/journal/19449208)

## Health Sciences

### Scraping [Annual Review of Public Health](https://www.annualreviews.org/content/journals/publhealth)

### Scraping [Lancet Public Health](https://www.thelancet.com/journals/lanpub/home)

### Scraping [Nature Reviews Cancer](https://www.nature.com/nrc/)

### Scraping [New England Journal of Medicine](https://www.nejm.org/)

### Scraping [The Lancet](https://www.thelancet.com/journals/lancet/home)

## Human Sciences

### Scraping [American Journal of Human Biology](https://onlinelibrary.wiley.com/journal/15206300?msockid=0525cb73d9a76a060b80df20d87e6b4b)

### Scraping [Annual Review of Anthropology](https://www.annualreviews.org/content/journals/anthro)

### Scraping [Journal of Human Evolution](https://www.sciencedirect.com/journal/journal-of-human-evolution)

## Linguistics, literature and arts

### Scraping [Applied Corpus Linguistics](https://www.sciencedirect.com/journal/applied-corpus-linguistics)

### Scraping [Journal of Aesthetic Education](https://press-jae.scholasticahq.com/)

### Scraping [Journal of English Linguistics](https://journals.sagepub.com/home/eng)

### Scraping [Modern Language Review](https://www.mhra.org.uk/journals/MLR)

### Scraping [PMLA: Publications of the Modern Language Association](https://www.mla.org/Publications/Journals/PMLA)